In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from transforms import imageTransforms
from search import ImageSearch
from analyzer import img2vec, image_from_url

#### ref: https://github.com/WillKoehrsen/Data-Analysis/blob/master/widgets/Widgets-Overview.ipynb

In [4]:
# maximizing plot windows: https://stackoverflow.com/a/22418354/3543635
# plt.switch_backend('TkAgg')

In [59]:
from ipywidgets import Dropdown, interact, interact_manual, FloatSlider

In [12]:
db_type = 'testing'
images = [(1, 'https://picsum.photos/id/448/1024/768')]
imageSearch = ImageSearch(db_type, images)
imageSearch.thresh = 20  # to pass the following tests

assert(len(imageSearch.vecs) > 0)

In [88]:
transforms = {'None': (0, 0), 'crop': (0, 0.5), 'rotate': (0, 0), 'invert': (0, 0), 'mirror': (0, 0), 'BLUR': (0, 0), 
              'CONTOUR': (0, 0), 'DETAIL': (0, 0), 'EDGE_ENHANCE': (0, 0), 'EDGE_ENHANCE_MORE': (0, 0), 'EMBOSS': (0, 0), 
              'FIND_EDGES': (0, 0), 'SMOOTH': (0, 0), 'SMOOTH_MORE': (0, 0), 'SHARPEN': (0, 0), 'GaussianBlur': (0, 2), 
              'UnsharpMask': (0, 0), 'MedianFilter': (0, 0), 'MinFilter': (0, 0), 'MaxFilter': (0, 0), 'ModeFilter': (0, 0)}
tf = 'None'

In [21]:
i = 1
image_url = images[0][1]

In [102]:
def get_transform(image, tf, param):
    imageTransformed = imageTransforms(image, type=tf, param=param)
    imageTransformedVec = img2vec(imageTransformed, type='image')

    ret = imageSearch.search(imageTransformedVec)            
    print(f'{i}=>{ret[0]}, {tf}: {ret[1]}')

    if ret[0] is not None:
        ret_image = image_from_url(images[ret[0]-1][1])['image']
    else:
        ret_image = None
    
    return imageTransformed, ret_image, ret
    
def plot_transforms(image, imageTransformed, ret_image, ret, tf):
    f, ax = plt.subplots(nrows=3, ncols=1, figsize=(10, 20))
    f.tight_layout()
    
    ax[0].imshow(image)
    ax[0].set_title('original image')
    ax[1].imshow(imageTransformed)
    ax[1].set_title(f'{tf} image')
    if ret_image is not None:
        ax[2].imshow(ret_image)
        ax[2].set_title(f'similar image: {ret[1]:.2f}')

In [103]:
image = image_from_url(image_url)['image']

In [104]:
transformDropdown = Dropdown(
    options=list(transforms.keys()),
    description='Transform: ',
    value='None'
)
paramSlider = FloatSlider(
    value=0,
    min=transforms['None'][0],
    max=transforms['None'][1],
    description='Param: '
)

def on_update_transform_widget(*args):
    paramSlider.min = transforms[transformDropdown.value][0]
    paramSlider.max = transforms[transformDropdown.value][1]

transformDropdown.observe(on_update_transform_widget, 'value')

In [106]:
# @interact
def do_the_interactive_thing(transformDropdown, paramSlider):
    imageTransformed, ret_image, ret = get_transform(image, transformDropdown, paramSlider)
    plot_transforms(image, imageTransformed, ret_image, ret, tranformDropdown.value)
    
interact(do_the_interactive_thing, transformDropdown=transformDropdown, paramSlider=paramSlider)

interactive(children=(Dropdown(description='Transform: ', index=1, options=('None', 'crop', 'rotate', 'invert'…

<function __main__.do_the_interactive_thing(transformDropdown, paramSlider)>